# Teleoperation

In this example we'll control the Jetbot remotely with a gamepad controller connected to our web browser machine.

### Create gamepad controller

The first thing we want to do is create an instance of the ``Controller`` widget, which we'll use to drive our robot.
The ``Controller`` widget takes a ``index`` parameter, which specifies the number of the controller.  This is useful in case you
have multiple controllers attached, or some gamepads *appear* as multiple controllers.  To determine the index
of the controller you're using,

1. Visit [http://html5gamepad.com](http://html5gamepad.com).  
2. Press buttons on the gamepad you're using
3. Remember the ``index`` of the gamepad that is responding to the button presses

Next, we'll create and display our controller using that index.

In [21]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)

Controller()

Even if the index is correct, you may see the text ``Connect gamepad and press any button``.  That's because the gamepad hasn't
registered with this notebook yet.  Press a button and you should see the gamepad widget appear above.

### Connect gamepad controller to robot motors

Now, even though we've connected our gamepad, we haven't yet attached the controls to our robot!  The first, and most simple control
we want to attach is the motor control.  We'll connect that to the left and right vertical axes using the ``dlink`` function.  The
``dlink`` function, unlike the ``link`` function, allows us to attach a transform between the ``source`` and ``target``.  Because
the controller axes are flipped from what we think is intuitive for the motor control, we'll use a small *lambda* function to
negate the value.

> WARNING: This next cell will move the robot if you touch the gamepad controller axes!

In [23]:
from jetbot import Robot
import traitlets

robot = Robot()

left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)

Awesome! Our robot should now respond to our gamepad controller movements.  Now we want to view the live video feed from the camera!

### Create and display Image widget

First, let's display an ``Image`` widget that we'll use to show our live camera feed.  We'll set the ``height`` and ``width``
to just 300 pixels so it doesn't take up too much space.

> FYI: The height and width only effect the rendering on the browser side, not the native image resolution before network transport from robot to browser.

In [24]:
image = widgets.Image(format='jpeg', width=300, height=300)

display(image)

Image(value=b'', format='jpeg', height='300', width='300')

### Create camera instance

Well, right now there's no image presented, because we haven't set the value yet!  We can do this by creating our ``Camera``
class and attaching the ``value`` attribute of the camera to the ``value attribute of the image.

First, let's create the camera instance, we call the ``instance`` method which will create a new camera
if it hasn't been created yet.  If once already exists, this method will return the existing camera.

In [25]:
from jetbot import Camera

camera = Camera.instance()

### Connect Camera to Image widget

Our camera class currently only produces values in BGR8 (blue, green, red, 8bit) format, while our image widget accepts values in compressed *JPEG*.
To connect the camera to the image we need to insert the ``bgr8_to_jpeg`` function as a transform in the link.  We do this below

In [26]:
from jetbot import bgr8_to_jpeg

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

You should now see the live video feed shown above!

> REMINDER:  You can right click the output of a cell and select ``Create New View for Output`` to display the cell in a separate window.

### Stop robot if network disconnects

You can drive your robot around by looking through the video feed. But what if your robot disconnects from Wifi?  Well, the motors would keep moving and it would keep trying to stream video and motor commands.  Let's make it so that we stop the robot and unlink the camera and motors when a disconnect occurs.

In [ ]:
from jetbot import Heartbeat


def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        left_link.unlink()
        right_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)

# attach the callback function to heartbeat status
heartbeat.observe(handle_heartbeat_status, names='status')

If the robot disconnects from the internet you'll notice that it stops.  You can then re-connect the camera and motors by re-creating the links with the cell below

In [ ]:
# only call this if your robot links were unlinked, otherwise we'll have redundant links which will double
# the commands transfered

left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

### Save snapshots with gamepad button

Now, we'd like to be able to save some images from our robot.  Let's make it so the right bumper (index 5) saves a snapshot of the current live image.  We'll save the images in the ``snapshots/`` directory, with a name that is guaranteed to be unique using the ``uuid`` python package.  We use the ``uuid1`` identifier, because this also encodes the date and MAC address which we might want to use later.

In [27]:
import uuid
import subprocess

subprocess.call(['mkdir', '-p', 'snapshots'])

snapshot_image = widgets.Image(format='jpeg', width=300, height=300)

def save_snapshot(change):
    # save snapshot when button is pressed down
    if change['new']:
        file_path = 'snapshots/' + str(uuid.uuid1()) + '.jpg'
        
        # write snapshot to file (we use image value instead of camera because it's already in JPEG format)
        with open(file_path, 'wb') as f:
            f.write(image.value)
            
        # display snapshot that was saved
        snapshot_image.value = image.value


controller.buttons[5].observe(save_snapshot, names='value')

display(widgets.HBox([image, snapshot_image]))
display(controller)

Controller(axes=(Axis(value=0.0), Axis(value=0.0), Axis(value=0.0), Axis(value=0.0), Axis(value=0.0), Axis(val…

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2, time
from jetpot import bgr8_to_jpeg

### Conclusion

That's it for this example, have fun!

In [ ]:
def preprocessing(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    gray = cv2.GaussianBlur(gray, (7,7),0)
    return gray

def thresholding(img_gray):
    _, img_th = cv2.threshold(img_gray,np.average(img_gray)-40,255,cv2.THRESH_BINARY)
    img_th2 = cv2.adaptiveThreshold(img_gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,21,15)
    img_th3 = np.bitwise_and(img_th, img_th2)
    img_th4 = cv2.subtract(img_th2, img_th3)
    for i in range(5):
        img_th4 = cv2.medianBlur(img_th4, 5)
    return img_th4

def mask_roi(img_th, roi):
    mask = np.zeros_like(img_th)
    cv2.fillPoly(mask, np.array([roi], np.int32), 255)
    masked_image = cv2.bitwise_and(img_th, mask)
    return masked_image

def drawContours(img_rgb, contours):
    for cnt in contours:
        area = cv2.contourArea(cnt)
        cv2.drawContours(img_rgb, [cnt], 0, (255,0,0), 1)
    return img_rgb

def approximationContour(img, contours, e=0.02):
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        epsilon = e*cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        cv2.drawContours(img, [approx], 0, (0,255,255), 2)
    return img

def rectwithname(img, contours, e=0.02):
    result = img.copy()
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        epsilon = e*cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        cv2.rectangle(result,(x,y),(x+w,y+h),(255,0,255),2)
    return result

def find_midptr(contours):
    center_ptrs = []
    e=0.01
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        center_ptr = [y, x + 0.5*w,]
        center_ptrs.append(center_ptr)
    center_ptrs = np.array(center_ptrs)
    return center_ptrs

def find_midlane(center_ptrs, center_image_point):
    L2_norm = np.linalg.norm((center_ptrs - center_image_point), axis=1, ord=2)
    loc = np.where(L2_norm==L2_norm.min())[0][0]
    midlane = center_ptrs[loc]
    return midlane

def find_degree(center_image_point, midlane):
    return 57.2958*np.arctan((midlane[1] - center_image_point[1])/(center_image_point[0] - midlane[0]))

In [ ]:
width = 224
height = 224
camera = Camera.instance()
input_image = widgets.Image(format='jpeg', width=width, height=height)
result1 = widgets.Image(format='jpeg', width=width, height=height)
result2 = widgets.Image(format='jpeg', width=width, height=height)
result3 = widgets.Image(format='jpeg', width=width, height=height)
result4 = widgets.Image(format='jpeg', width=width, height=height)
image_box = widgets.HBox([input_image, result1, result2, result3, result4], layout=widgets.Layout(align_self='center'))
display(image_box)
# display(result)

In [ ]:
count = 0
while True:
    img = camera.value
    img_gray = preprocessing(img)
    img_th = thresholding(img_gray)
    roi = [(0, height),(0, height/2-30), (width, height/2-30),(width, height),]
    img_roi = mask_roi(img_th, roi)
    
    kernel = np.ones((5,3),np.uint8)
    img_cl = cv2.morphologyEx(img_roi,cv2.MORPH_CLOSE, np.ones((5,5),np.uint8),iterations=4)
    img_op = cv2.morphologyEx(img_cl,cv2.MORPH_OPEN, np.ones((5,5),np.uint8),iterations=3)
    
    cannyed_image = cv2.Canny(img_op, 300, 500)
    contours, _ = cv2.findContours(cannyed_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    img_approx = approximationContour(img, contours, e=0.02)
    img_approx_rect = rectwithname(img, contours, e=0.01)  
    
    center_ptrs = find_midptr(contours)
    
    center_image_point = [height-1, width/2-1]
        
    midlane = find_midlane(center_ptrs, center_image_point)
    seta = find_degree(center_image_point, midlane)
    
    cv2.line(img,(int(center_image_point[1]), int(center_image_point[0])),(int(midlane[1]),int(midlane[0])),(0,0,255),3)
    cv2.putText(img, f'{seta}', (int(midlane[1]), int(midlane[0])-5), cv2.FONT_HERSHEY_COMPLEX, 0.5,(255, 0, 0), 1)
    
    result_img1 = img_th
    result_img2 = img_cl
    result_img3 = img_op
    result_img4 = img
    
    #show results
    result_imgs = [result_img1, result_img2, result_img3, result_img4]
    result_values = [result1, result2, result3, result4]
    for result_img, result_value in zip(result_imgs, result_values):
         if len(result_img.shape)==2:
             result_img = np.stack((result_img,)*3,2)
        result_value.value = bgr8_to_jpeg(result_img)
    input_image.value = bgr8_to_jpeg(img_gray)
    
    if count ==1000:
        break
    else:
        count = count +1
         print(count, end='  ')
        time.sleep(0.1)